In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as stats
import math
import os
%matplotlib inline


In [2]:
# Sets path for data files and reads them into a dictionary
path = '~/GIT/PSU_ME_Stuff/ME365/Labs/Lab4/'
files = ['Lab4_420G4.xlsx','Lab4_440AG4.xlsx','Lab4_Nimark_300G4.xlsx','Lab4_Custom_455G4.xlsx']
dfKeys = ['dat420','dat440','dat455','dat300']

dfs = {}
for df,filename in zip(dfKeys,files):
    dfs[df]=(pd.read_excel(path+filename))

In [3]:
dfs

{'dat420':    Sample  Thickness  %Reduction     1   2     3   4 scale
 0       1      0.258         NaN  93.5  95  95.0  93     B
 1       2      0.237    8.139535  93.5  99  98.5  99     B
 2       3      0.213   17.441860  21.0  20  21.0  20     C
 3       4      0.154   40.310078  26.0  26  26.0  26     C
 4       5      0.106   58.914729  30.0  30  30.0  30     C,
 'dat440':    Sample  Thickness  %Reduction   1   2   3   4 scale
 0       5      0.084         NaN  96  97  97  97     B
 1       4      0.077    8.333333  22  22  22  22     C
 2       3      0.065   22.619048  26  27  25  25     C
 3       2      0.052   38.095238  28  27  28  31     C
 4       1      0.040   52.380952  29  29  30  29     C,
 'dat455':    Sample  Thickness  %Reduction   1   2   3   4 scale
 0       1      0.080         NaN  32  32  33  31     C
 1       2      0.072       34.00  32  32  33  34     C
 2       3      0.065       18.75  35  34  35  35     C
 3       4      0.057       28.75  32  33  33  3

In [4]:
# Convert all Rockwell B to C for 440 data
dfs['dat440'].iloc[0,3] = 18
dfs['dat440'].iloc[0,4:7] = 20

# Convert all Rockwell B to C for 
def interpolate(x1,x2,y1,y2,X):
    
    return y1 + (y1 - y2)*(X - x1)/(x1 - x2)

x1 = 93
x2 = 94
X = 93.5
y1 = 13
y2 = 15
dfs['dat420'].iloc[0:2,3] = interpolate(x1,x2,y1,y2,X)
x1 = 98
x2 = 99
X = 98.5
y1 = 21
y2 = 22
dfs['dat420'].iloc[1,5] = interpolate(x1,x2,y1,y2,X)
dfs['dat420'].iloc[0,4:6] = 16.
dfs['dat420'].iloc[0,6] = 13.
dfs['dat420'].iloc[1,4] = 22.
dfs['dat420'].iloc[1,6] = 22.
dfs['dat420'].iloc[0:2,7] = 'C'
dfs['dat440'].iloc[0,7] = 'C'

In [5]:
# Retrieve t-stat value from the "chart" TINV(a,v) 95% confidence interval
alpha = .05
t = stats.t.ppf(1-alpha/2,df=6)
print(t)

2.4469118487916806


In [6]:
for df in dfKeys:
 
    print(df,'\n',dfs[df],'\n')

dat420 
    Sample  Thickness  %Reduction     1     2     3     4 scale
0       1      0.258         NaN  14.0  16.0  16.0  13.0     C
1       2      0.237    8.139535  14.0  22.0  21.5  22.0     C
2       3      0.213   17.441860  21.0  20.0  21.0  20.0     C
3       4      0.154   40.310078  26.0  26.0  26.0  26.0     C
4       5      0.106   58.914729  30.0  30.0  30.0  30.0     C 

dat440 
    Sample  Thickness  %Reduction   1   2   3   4 scale
0       5      0.084         NaN  18  20  20  20     C
1       4      0.077    8.333333  22  22  22  22     C
2       3      0.065   22.619048  26  27  25  25     C
3       2      0.052   38.095238  28  27  28  31     C
4       1      0.040   52.380952  29  29  30  29     C 

dat455 
    Sample  Thickness  %Reduction   1   2   3   4 scale
0       1      0.080         NaN  32  32  33  31     C
1       2      0.072       34.00  32  32  33  34     C
2       3      0.065       18.75  35  34  35  35     C
3       4      0.057       28.75  32  33 

In [10]:
# This mess creates a function that does the rest of the statistcal calculations and writes it to DataFrames
def trueMean(dfs,n1,n2):
    alpha = .05
    t = stats.t.ppf(1.-alpha/2.,df=6)
    #print t
    v1 = n1-1
    v2 = n2-1
    df = n1+n2-2
    v = v1+v2
    for df in dfs.keys():
        df = dfs[df]
        df['mean_hardness'] = 0.
        df['std'] = 0.
        
        #df['true_mean+'] = 0.
        #df['true_mean-'] = 0.
        #df['meanDiff'] =''
       
        for i in range(5):
            df.loc[i,'mean_hardness'] = sum(df.iloc[i,3:7])/4.   
            df.loc[i,'std'] = df.iloc[i,3:7].std()
            if i >=1:
                
                df.loc[i,'Sx'] = np.sqrt((v1*(df.loc[0,'std'])**2+v2*(df.loc[i,'std'])**2)/(v))
            else:
                #print(df)
                df.loc[i,'Sx'] = 'NaN'
        for i in range(5):
            
            if i >= 1:
                df.loc[i,'true_mean+'] = df.loc[0,'mean_hardness']-df.loc[i,'mean_hardness'] + t*df.loc[i,'Sx']*np.sqrt(1/n1+1/n2)
                df.loc[i,'true_mean-'] = df.loc[0,'mean_hardness']-df.loc[i,'mean_hardness'] - t*df.loc[i,'Sx']*np.sqrt(1/n1+1/n2)
                #print(df.loc[0,'mean_hardness'],df.loc[i,'mean_hardness'])
            else:
                df.loc[i,'true_mean+'] = 'NaN'
                df.loc[i,'true_mean-'] = 'NaN'
                df.loc[i,'meanDiff'] ='NaN'
            if df.loc[i,'true_mean+'] != 'NaN' and df.loc[i,'true_mean-'] != 'NaN':
                if df.loc[i,'true_mean+'] < 0 and df.loc[i,'true_mean-'] < 0 or df.loc[i,'true_mean+'] > 0 and df.loc[i,'true_mean-'] > 0:
                    df.loc[i,'meanDiff'] = 'Yes'
                
                else:
                    df.loc[i,'meanDiff'] = 'No'
    return('Statiscal Analysis Successful\n\n')

In [11]:
# This 
#dfs = [dat420,dat440,dat455,dat300]
print(trueMean(dfs,4,4))
#print trueMean(dfs,5.,5.)
print('Type 420 G4\n\n',dfs['dat420'])
print('\nType 440 G4\n\n',dfs['dat440'])
print('\nType 455 G4\n\n',dfs['dat455'])
print('\nType300 G4\n\n',dfs['dat300'])


Statiscal Analysis Successful


Type 420 G4

    Sample  Thickness  %Reduction     1     2     3     4 scale  mean_hardness  \
0       1      0.258         NaN  14.0  16.0  16.0  13.0     C         14.750   
1       2      0.237    8.139535  14.0  22.0  21.5  22.0     C         19.875   
2       3      0.213   17.441860  21.0  20.0  21.0  20.0     C         20.500   
3       4      0.154   40.310078  26.0  26.0  26.0  26.0     C         26.000   
4       5      0.106   58.914729  30.0  30.0  30.0  30.0     C         30.000   

        std       Sx true_mean+ true_mean- meanDiff  
0  1.500000      NaN        NaN        NaN      NaN  
1  3.923752  2.97034  0.0143644   -10.2644       No  
2  0.577350  1.13652   -3.78357   -7.71643      Yes  
3  0.000000  1.06066   -9.41482   -13.0852      Yes  
4  0.000000  1.06066   -13.4148   -17.0852      Yes  

Type 440 G4

    Sample  Thickness  %Reduction   1   2   3   4 scale  mean_hardness  \
0       5      0.084         NaN  18  20  20  20     C 

In [9]:
#path = '~/GIT/PSU_ME_Stuff/ME365/Labs/Lab4/'
#files = ['Lab4_420G4_stat.xlsx','Lab4_440AG4_stat.xlsx','Lab4_Custom_455G4_stat.xlsx','Lab4_Nimark_300G4_stat.xlsx']
#for df,fileName in zip(dfs.keys(),files):
#    dfs[df].to_excel(path + fileName)
